Find the leading scorer by zone on the court. Plot the zones and players' names.

In [ ]:
import os
import sys
import pandas as pd
from matplotlib import pyplot as plt

sys.path.append("../utils/")
from draw_court import draw_court

# %matplotlib

plt.close("all")

In [ ]:
data_dir = "~/Documents/euroleague_api/notebooks/data"

In [ ]:
shot_df = pd.read_csv(os.path.join(data_dir, "shot_data_E2023.csv"))

In [ ]:
def get_leading_scorer_by_zone(df):
    top_players_by_zone_df = df.groupby(["ZONE", "PLAYER"])["POINTS"].sum().reset_index()
    pidx = top_players_by_zone_df.groupby('ZONE')['POINTS'].idxmax()
    top_players = top_players_by_zone_df.loc[pidx]
    return top_players

In [ ]:
def plot_leading_scorers_by_zone(shot_df, lead_scorers_df):
    plt.figure()
    draw_court()
    plt.xlim([-800, 800])
    plt.ylim([-200, 1300])
    for zone in lead_scorers_df.ZONE.unique():
        if zone not in [" ", "J"]:
            zone_df = shot_df[shot_df["ZONE"] == zone]
            y_annot = zone_df['COORD_Y'].median()
            if zone in ["B", "D", "F", "H"]:
                x_annot = zone_df['COORD_X'].median() - 150
            elif zone == "A":
                x_annot = -70
                y_annot = -70
            else:
                x_annot = zone_df['COORD_X'].median()
            player = lead_scorers_df.loc[lead_scorers_df["ZONE"] == zone, "PLAYER"].iloc[0]
            last, first = player.split(", ")
            player = f"{last} {first[:1]}."
            plt.annotate(player, (x_annot, y_annot), fontsize=8)
            plt.plot(zone_df['COORD_X'], zone_df['COORD_Y'], 'o', mfc='none', zorder=0)
    plt.show()
    return

In [ ]:
top_players = get_leading_scorer_by_zone(shot_df)
top_players

In [ ]:
plot_leading_scorers_by_zone(shot_df, top_players)

## PAO's Leading scorers by zone

In [ ]:
pao_df = shot_df[shot_df["TEAM"] == "PAN"]

In [ ]:
top_players_pao_df = get_leading_scorer_by_zone(pao_df)
top_players_pao_df

In [ ]:
plot_leading_scorers_by_zone(pao_df, top_players_pao_df)